# Importacion de librerias

In [0]:
!pip install wbgapi
import wbgapi as wb
import pandas as pd
import numpy as np
import time
import pandas as pd
import io
import requests
from pandas import json_normalize

     |████████████████████████████████| 701 kB 7.0 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


# Creacion de los esquemas

In [0]:
%sql 
CREATE SCHEMA IF NOT EXISTS transformed_data;
CREATE SCHEMA IF NOT EXISTS analysis_data;

## Funcion para la extraccion de datos del Banco mundial de datos

In [0]:
#Funciones
#banco_mundial
def getDataFrameDatabank(countries_list,dataDicc):
    dataset = np.array([])
    for i in dataDicc.items():
        test = wb.data.fetch(i[0], countries_list)
        time.sleep(1)
        list_countries = []
        list_years = []
        list_data = []
        for a in test:
            list_countries.append(a['economy'])
            list_years.append((a['time'][2::]))
            list_data.append((a['value']))
        add = {'title':i[1],
            'economy':list_countries,
            'time':list_years,
            'value':list_data}
        dataset = np.append(dataset, add)

    #validation data dimentions

    for i in range(0,len(dataset)):
        print('dimention of data list: '+str(dataset[i]['title']))
        print(len(dataset[i]['value']))

    for i in range(0,len(dataset)-1):
        if (dataset[i]['time'] == dataset[i+1]['time']):
            print('is equal times')
        if (dataset[i]['economy'] == dataset[i+1]['economy']):
            print('is equal countries')

    # dataframe export
    dataframe = pd.DataFrame()
    dataframe['country'] = dataset[0]['economy'] 
    dataframe['year'] = dataset[0]['time'] 
    for i in range(0,len(dataset)):
        dataframe[dataset[i]['title']] = dataset[i]['value'] 
    return dataframe

## Lista de codigos de paises sobre los que se abordara el trabajo.

In [0]:
countries_list = ['ARG','BRA','COL','CAN','CHL','MEX','ECU','USA','URY', 'PAN','VEN',
                  'DEU','BEL','ESP','FRA','NLD','GBR','ITA','CHE','UKR', 'RUS',
                  'AGO','EGY','CMR','NGA','ZAF','ETH',
                  'CHN','JPN','KOR','MYS','IND','AFG']

## Tablas

### Esperanza de vida

In [0]:
dataEsp = {'SP.DYN.LE00.FE.IN':'Life_expectancy_at_birth_female_years',
            'SP.DYN.LE00.MA.IN':'Life_expectancy_at_birth_male_years'}

In [0]:
esperanza_vida = getDataFrameDatabank(countries_list, dataEsp)

dimention of data list: Life_expectancy_at_birth_female_years
2046
dimention of data list: Life_expectancy_at_birth_male_years
2046
is equal times
is equal countries


In [0]:
spark_esp = spark.createDataFrame(esperanza_vida)
spark_esp

Out[6]: DataFrame[country: string, year: string, Life_expectancy_at_birth_female_years: double, Life_expectancy_at_birth_male_years: double]

In [0]:
spark_esp.write.mode('overwrite').option("mergeSchema", "true").saveAsTable('transformed_data.esperanza_vida')

### Eje salud

In [0]:
dataSalud = {#Sociales
            'SH.STA.BRTC.ZS':'Births_attended_by_skilled_health_staff_percentage_of_total',
            #economicos
            'SH.XPD.CHEX.GD.ZS':'Current_health_expenditure_percentage_of_GDP',
            'SH.XPD.CHEX.PC.CD':'Current_health_expenditure_per_capita_current_USdollar',
            'SH.XPD.EHEX.PC.CD':'External_health_expenditure_per_capita_current_USdollar',
            'SH.XPD.GHED.GD.ZS':'Domestic_general_government_health_expenditure_percentage_of_GDP',
            'SH.XPD.GHED.PC.CD':'Domestic_general_government_health_expenditure_per_capita_current_USdollar',
            'SH.XPD.PVTD.PC.CD':'Domestic_private_health_expenditure_per_capita_current_USdollar',
            #riesgo_de_muerte
            'SH.MMR.RISK.ZS':'Lifetime_risk_of_maternal_death_percentage',
            'SP.DYN.LE00.FE.IN':'Life_expectancy_at_birth_female_years',
            'SP.DYN.LE00.MA.IN':'Life_expectancy_at_birth_male_years',
            #Mortalidad
            'SH.DTH.NMRT':'Number_of_neonatal_deaths',
            'SH.DYN.NMRT':'Mortality_rate_neonatal_per_1000_live_births',
            'SP.DYN.IMRT.FE.IN':'Mortality_rate_infant_female_per_1000_live_births',
            'SP.DYN.IMRT.MA.IN':'Mortality_rate_infant_male_per_1000_live_births',
            'SH.DTH.IMRT':'Number_of_infant_deaths',
            'SH.DYN.MORT.FE':'Mortality_rate_under_5_female_per_1000_live_births',
            'SH.DYN.MORT.MA':'Mortality_rate_under_5_male_per_1000_live_births',
            'SH.DTH.0509':'Number_of_deaths_ages_5_9_years',
            'SH.DTH.1014':'Number_of_deaths_ages_10_14_years',
            'SH.DTH.1519':'Number_of_deaths_ages_15_19_year',
            'SH.DTH.2024':'Number_of_deaths_ages_20_24_years',
            'SP.DYN.AMRT.FE':'Mortality_rate_adult_female_per_1000_female_adults',
            'SP.DYN.AMRT.MA':'Mortality_rate_adult_male_per_1000_male_adults',
            #maternas
            'SH.STA.MMRT.NE':'Maternal_mortality_ratio_national_estimate_per_100000_live_birth',
            'SH.MMR.DTHS':'Number_of_maternal_deaths',
            #Suicidio
            'SH.STA.SUIC.FE.P5':'Suicide_mortality_rate_female_per_100000_female_population',
            'SH.STA.SUIC.MA.P5':'Suicide_mortality_rate_male_per_100000_male_population',
            #fertilidad
            'SP.ADO.TFRT':'Adolescent_fertility_rate_births_per_1000_women_ages_15_19',
            'SP.DYN.TFRT.IN':'Fertility_rate_total_births_per_woman'}


In [0]:
eje_salud = getDataFrameDatabank(countries_list, dataSalud)
eje_salud


dimention of data list: Births_attended_by_skilled_health_staff_percentage_of_total
2046
dimention of data list: Current_health_expenditure_percentage_of_GDP
2046
dimention of data list: Current_health_expenditure_per_capita_current_USdollar
2046
dimention of data list: External_health_expenditure_per_capita_current_USdollar
2046
dimention of data list: Domestic_general_government_health_expenditure_percentage_of_GDP
2046
dimention of data list: Domestic_general_government_health_expenditure_per_capita_current_USdollar
2046
dimention of data list: Domestic_private_health_expenditure_per_capita_current_USdollar
2046
dimention of data list: Lifetime_risk_of_maternal_death_percentage
2046
dimention of data list: Life_expectancy_at_birth_female_years
2046
dimention of data list: Life_expectancy_at_birth_male_years
2046
dimention of data list: Number_of_neonatal_deaths
2046
dimention of data list: Mortality_rate_neonatal_per_1000_live_births
2046
dimention of data list: Mortality_rate_infan

,country,year,Births_attended_by_skilled_health_staff_percentage_of_total,Current_health_expenditure_percentage_of_GDP,Current_health_expenditure_per_capita_current_USdollar,External_health_expenditure_per_capita_current_USdollar,Domestic_general_government_health_expenditure_percentage_of_GDP,Domestic_general_government_health_expenditure_per_capita_current_USdollar,Domestic_private_health_expenditure_per_capita_current_USdollar,Lifetime_risk_of_maternal_death_percentage,...,Number_of_deaths_ages_15_19_year,Number_of_deaths_ages_20_24_years,Mortality_rate_adult_female_per_1000_female_adults,Mortality_rate_adult_male_per_1000_male_adults,Maternal_mortality_ratio_national_estimate_per_100000_live_birth,Number_of_maternal_deaths,Suicide_mortality_rate_female_per_100000_female_population,Suicide_mortality_rate_male_per_100000_male_population,Adolescent_fertility_rate_births_per_1000_women_ages_15_19,Fertility_rate_total_births_per_woman
0,AFG,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,2020,61.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12079.0,13913.0,185.146,230.214,NaN,NaN,NaN,NaN,57.5090,4.176
2,AFG,2019,NaN,13.242202,65.806030,8.167607,1.084431,5.388990,52.249440,NaN,...,12081.0,13531.0,188.839,233.884,NaN,NaN,3.6,4.6,61.3250,4.321
3,AFG,2018,58.8,14.126743,69.998604,13.854496,0.549220,2.721409,53.422707,NaN,...,12044.0,13135.0,192.532,237.554,NaN,NaN,3.5,4.6,65.1410,4.473
4,AFG,2017,53.4,12.620817,65.706024,12.760199,0.643134,3.348260,49.597570,3.063593,...,11908.0,12631.0,198.184,240.314,NaN,7700.0,3.4,4.6,68.9570,4.633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2041,ARG,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,145.115,251.743,NaN,NaN,NaN,NaN,60.8324,3.068
2042,ARG,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,146.321,250.957,NaN,NaN,NaN,NaN,60.9702,3.079
2043,ARG,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,147.528,250.171,NaN,NaN,NaN,NaN,61.1080,3.090
2044,ARG,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,151.879,252.620,NaN,NaN,NaN,NaN,61.2480,3.100


In [0]:
#convert the pandas dataframe to a pyspark dataframe and save it as a table:
spark_sa = spark.createDataFrame(eje_salud)



In [0]:
display(eje_salud)

country,year,Births_attended_by_skilled_health_staff_percentage_of_total,Current_health_expenditure_percentage_of_GDP,Current_health_expenditure_per_capita_current_USdollar,External_health_expenditure_per_capita_current_USdollar,Domestic_general_government_health_expenditure_percentage_of_GDP,Domestic_general_government_health_expenditure_per_capita_current_USdollar,Domestic_private_health_expenditure_per_capita_current_USdollar,Lifetime_risk_of_maternal_death_percentage,Life_expectancy_at_birth_female_years,Life_expectancy_at_birth_male_years,Number_of_neonatal_deaths,Mortality_rate_neonatal_per_1000_live_births,Mortality_rate_infant_female_per_1000_live_births,Mortality_rate_infant_male_per_1000_live_births,Number_of_infant_deaths,Mortality_rate_under_5_female_per_1000_live_births,Mortality_rate_under_5_male_per_1000_live_births,Number_of_deaths_ages_5_9_years,Number_of_deaths_ages_10_14_years,Number_of_deaths_ages_15_19_year,Number_of_deaths_ages_20_24_years,Mortality_rate_adult_female_per_1000_female_adults,Mortality_rate_adult_male_per_1000_male_adults,Maternal_mortality_ratio_national_estimate_per_100000_live_birth,Number_of_maternal_deaths,Suicide_mortality_rate_female_per_100000_female_population,Suicide_mortality_rate_male_per_100000_male_population,Adolescent_fertility_rate_births_per_1000_women_ages_15_19,Fertility_rate_total_births_per_woman
AFG,2021,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
AFG,2020,61.8,null,null,null,null,null,null,null,66.744,63.708,42758.0,35.2,41.5,48.1,54151.0,54.3,61.4,3584.0,2483.0,12079.0,13913.0,185.146,230.214,null,null,null,null,57.509,4.176
AFG,2019,null,13.24220181,65.80603027,8.16760669,1.08443093,5.38899046,52.24943995,null,66.388,63.382,43747.0,36.1,42.9,49.6,55690.0,56.4,63.5,3348.0,2373.0,12081.0,13531.0,188.839,233.884,null,null,3.6,4.6,61.325,4.321
AFG,2018,58.8,14.12674332,69.99860382,13.85449585,0.54922014,2.72140895,53.42270666,null,66.026,63.047,44867.0,37.2,44.5,51.1,57231.0,58.7,65.8,3501.0,2425.0,12044.0,13135.0,192.532,237.554,null,null,3.5,4.6,65.141,4.473
AFG,2017,53.4,12.62081718,65.70602417,12.76019879,0.64313394,3.34826009,49.59757036,3.06359325193536,65.656,62.701,45875.0,38.2,46.1,52.8,58923.0,61.1,68.2,3672.0,2500.0,11908.0,12631.0,198.184,240.314,null,7700.0,3.4,4.6,68.957,4.633
AFG,2016,null,11.81856155,60.18867111,11.40444159,0.60012823,3.05628788,45.72794863,3.3309138614947,65.275,62.343,47047.0,39.3,47.7,54.6,60700.0,63.8,71.0,3856.0,2594.0,11583.0,11970.0,202.465,244.519,1194.0,8100.0,3.6,4.4,75.325,4.8
AFG,2015,50.5,10.10534763,58.90652847,9.68745171,0.52267283,3.04678667,46.17228869,3.6091255501105,64.877,61.965,48304.0,40.5,49.6,56.5,62703.0,66.7,73.9,4082.0,2724.0,10922.0,11169.0,206.746,248.724,null,8400.0,3.6,4.3,81.693,4.976
AFG,2014,45.2,9.52887058,59.00893402,12.97241822,0.47265655,2.92699537,43.1095203,4.21670381435676,64.453,61.563,49707.0,41.9,51.6,58.6,64844.0,69.9,77.1,4381.0,2890.0,9889.0,10096.0,211.026,252.93,null,9300.0,3.6,4.2,88.061,5.163
AFG,2013,null,8.80594063,55.03462982,12.61249264,0.44335812,2.77086252,39.65127422,4.53815117369347,63.999,61.133,51184.0,43.2,53.6,60.8,67099.0,73.2,80.4,4706.0,3074.0,8675.0,8965.0,215.307,257.135,null,9600.0,3.7,4.3,94.429,5.359
AFG,2012,39.9,7.89717579,51.44026184,10.89087882,0.34288704,2.233482,38.31590145,5.02302719536136,63.514,60.675,52710.0,44.6,55.7,63.0,69517.0,76.7,83.8,4984.0,3228.0,7467.0,7909.0,219.588,261.34,null,10000.0,3.7,4.3,100.797,5.562


In [0]:
spark_sa.write.mode('overwrite').option("mergeSchema", "true").saveAsTable('transformed_data.eje_salud')

### Eje Economia

In [0]:
dataEcon = {'NV.SRV.TOTL.ZS': 'porcentaje_gdp_servicios',
            'NV.AGR.TOTL.ZS': 'porcentaje_gdp_agricultura',
            'NV.IND.MANF.ZS': 'porcentaje_gdp_manufactura',
            'NV.IND.TOTL.ZS': 'porcentaje_gdp_industria',
            'NY.GDP.MKTP.CD': 'gdp_usd',
            'NY.GDP.PCAP.CD': 'gdp_percapita'}

eje_economico = getDataFrameDatabank(countries_list, dataEcon)


dimention of data list: porcentaje_gdp_servicios
2046
dimention of data list: porcentaje_gdp_agricultura
2046
dimention of data list: porcentaje_gdp_manufactura
2046
dimention of data list: porcentaje_gdp_industria
2046
dimention of data list: gdp_usd
2046
dimention of data list: gdp_percapita
2046
is equal times
is equal countries
is equal times
is equal countries
is equal times
is equal countries
is equal times
is equal countries
is equal times
is equal countries


In [0]:
spark_ec = spark.createDataFrame(eje_economico)



In [0]:
display(spark_ec)

country,year,porcentaje_gdp_servicios,porcentaje_gdp_agricultura,porcentaje_gdp_manufactura,porcentaje_gdp_industria,gdp_usd,gdp_percapita
AFG,2021,null,null,null,null,null,null
AFG,2020,54.6883422622417,26.8206001075115,7.67625685837314,14.0316990961127,2.01161373258206E10,516.747870807558
AFG,2019,55.4728067408306,25.7739707394105,7.04318108092508,14.0581123697768,1.87994507427823E10,494.179349886029
AFG,2018,59.4774229106653,22.0428967527575,6.16017721650239,13.3872469597976,1.80532285788878E10,485.668418729808
AFG,2017,58.3290500097748,26.4201990834448,3.53042246146676,10.0518739969063,1.87534696302586E10,516.679862210543
AFG,2016,59.0232164144095,25.7403140364582,4.1141966128347,10.4668076938201,1.81165624649088E10,512.012778129357
AFG,2015,53.2352934986503,20.6343227166798,11.4200060049012,22.1240424906926,1.9134211763859E10,556.007220861443
AFG,2014,52.9908650351499,22.1370413719574,11.477587555803,21.2296632181805,2.04971267701335E10,614.223342360392
AFG,2013,52.6827434733525,22.8106627361766,11.4998672561905,20.4446053132561,2.0146404996223E10,624.315454506615
AFG,2012,50.5793979236606,24.390873626371,12.4860851363953,21.1578071539751,1.99073170656666E10,638.845851607289


In [0]:
spark_ec.write.mode('overwrite').saveAsTable('transformed_data.eje_economico')

### Eje Ambiental

Fuente Bano mundial de datos

In [0]:
dataAmb = {'ER.H2O.INTR.PC':'Agua_dulce_per_cápita_en_m3'}

In [0]:
eje_ambiental = getDataFrameDatabank(countries_list, dataAmb)

dimention of data list: Agua_dulce_per_cápita_en_m3
2046


In [0]:
spark_amb = spark.createDataFrame(eje_ambiental)

In [0]:
display(spark_amb)

country,year,Agua_dulce_per_cápita_en_m3
AFG,2021,null
AFG,2020,null
AFG,2019,null
AFG,2018,1268.43051053427
AFG,2017,1299.03724396258
AFG,2016,null
AFG,2015,null
AFG,2014,null
AFG,2013,null
AFG,2012,1513.09098076471


In [0]:
spark_amb.write.mode('overwrite').saveAsTable('transformed_data.eje_ambiental')

Otras fuentes

In [0]:
# File location and type
file_location = "/FileStore/tables/Epi_des_nat.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
eje_ambiental_pais = df.toPandas()
eje_ambiental_pais.sort_values(by=['country'], inplace = True)
eje_ambiental_pais.reset_index(drop = True, inplace = True)

In [0]:
spark_amb_pais = spark.createDataFrame(eje_ambiental_pais)

In [0]:
display(spark_amb_pais)

country,EPI.new,EPI.old,EPI.change,EPI.rnk.new,EPI.rnk.old,%_Afectado
AFG,43.6,19.7,23.9,81,178,1.05929016894999
AGO,30.5,30.3,0.2,151,147,1.01176467635625
ARG,41.1,33.3,7.8,92,129,0.167189950512828
BEL,58.2,52.1,6.1,21,24,0.00169216806014966
BRA,43.6,38.2,5.4,81,92,0.482451027992547
CAN,50.0,46.0,4.0,49,52,0.0107432376669766
CHE,65.9,57.7,8.2,9,7,0.00388788412428912
CHL,46.7,39.9,6.8,65,83,0.260470805127432
CHN,28.4,17.0,11.4,160,180,7.95097147464036
CMR,30.2,32.2,-2.0,153,136,0.0723722046654969


In [0]:
spark_amb_pais.write.mode('overwrite').saveAsTable('transformed_data.eje_ambiental_pais')

### Eje Politica

In [0]:
#Aca va el csv que trae Ale
# File location and type
file_location = "/FileStore/tables/Paises_Politica_formateado.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
eje_politica = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
eje_politica.write.mode('overwrite').saveAsTable('transformed_data.eje_politica')

In [0]:
display(eje_politica)

Año,Pais,Codigo,Estimate,StdErr,NumSrc,Rank,Lower,Upper
1996,Afghanistan,AFG,"-1,29","0,34",2,"4,3",0,"27,42"
1998,Afghanistan,AFG,"-1,18","0,32",2,"8,02",0,"33,69"
2000,Afghanistan,AFG,"-1,27","0,35",2,"4,79",0,"30,85"
2002,Afghanistan,AFG,"-1,25","0,35",2,"4,76",0,"32,8"
2003,Afghanistan,AFG,"-1,34","0,27",3,"4,76",0,"19,05"
2004,Afghanistan,AFG,"-1,35","0,21",5,"6,4",0,"15,27"
2005,Afghanistan,AFG,"-1,45","0,21",5,"1,46",0,"13,17"
2006,Afghanistan,AFG,"-1,45","0,21",7,"1,95",0,"11,71"
2007,Afghanistan,AFG,"-1,61","0,23",6,"0,97",0,"6,8"
2008,Afghanistan,AFG,"-1,67","0,22",7,"0,49",0,"4,37"


### Eje Educacion

In [0]:
dataEdu = {'SE.ADT.LITR.ZS':'%_de_+15_anios_que_pueden_leer_y_escribir',
            'SE.SEC.CUAT.UP.ZS':'%_de_+25_anios_con_secundario_completo'
           }
            

In [0]:
eje_educacion = getDataFrameDatabank(countries_list, dataEdu)

dimention of data list: %_de_+15_anios_que_pueden_leer_y_escribir
2046
dimention of data list: %_de_+25_anios_con_secundario_completo
2046
is equal times
is equal countries


In [0]:
spark_ed = spark.createDataFrame(eje_educacion)
spark_ed


Out[32]: DataFrame[country: string, year: string, %_de_+15_anios_que_pueden_leer_y_escribir: double, %_de_+25_anios_con_secundario_completo: double]

In [0]:
display(spark_ed)

country,year,%_de_+15_anios_que_pueden_leer_y_escribir,%_de_+25_anios_con_secundario_completo
AFG,2021,37.266040802002,9.46131038665771
AFG,2020,null,null
AFG,2019,null,null
AFG,2018,null,null
AFG,2017,null,null
AFG,2016,null,null
AFG,2015,null,null
AFG,2014,null,null
AFG,2013,null,null
AFG,2012,null,null


In [0]:
spark_ed.write.mode('overwrite').option("mergeSchema", "true").saveAsTable('transformed_data.eje_educacion')